In [5]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Define constants
data_dir = 'synthetic'  # Specify the path to your data directory
image_size = (128, 128)  # Define the desired image size
num_classes = 1  # Number of output classes (child height)
batch_size = 32
epochs = 20

In [ ]:
# Function to load images, depth maps, and labels
def load_data(data_dir):
    images = []
    depths = []
    labels = []

    for file in os.listdir(data_dir):
        if file.endswith('.png') and file.startswith('scene'):
            # Load image
            image_path = os.path.join(data_dir, file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, image_size)  # Resize image
            images.append(image)

            # Load corresponding depth map
            depth_file = file.replace('scene', 'depth')
            depth_path = os.path.join(data_dir, depth_file)
            depth = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
            depth = cv2.resize(depth, image_size)  # Resize depth map
            depths.append(depth)

            # Load corresponding label
            label_file = file.replace('.png', '.txt')
            label_path = os.path.join(data_dir, label_file)
            with open(label_path, 'r') as f:
                label = float(f.read().strip().split(':')[1])
            labels.append(label)

    return np.array(images), np.array(depths), np.array(labels)

# Load data
images, depths, labels = load_data(data_dir)

In [2]:
# Split data into training and testing sets
images_train, images_test, depths_train, depths_test, labels_train, labels_test = train_test_split(
    images, depths, labels, test_size=0.2, random_state=42)

# Define CNN model
def create_model():
    inputs_img = layers.Input(shape=image_size + (3,))
    inputs_depth = layers.Input(shape=image_size + (1,))

    # Convolutional layers for image
    conv_img = layers.Conv2D(32, (3, 3), activation='relu')(inputs_img)
    conv_img = layers.MaxPooling2D((2, 2))(conv_img)
    conv_img = layers.Conv2D(64, (3, 3), activation='relu')(conv_img)
    conv_img = layers.MaxPooling2D((2, 2))(conv_img)
    conv_img = layers.Conv2D(128, (3, 3), activation='relu')(conv_img)
    conv_img = layers.MaxPooling2D((2, 2))(conv_img)
    flatten_img = layers.Flatten()(conv_img)

    # Convolutional layers for depth map
    conv_depth = layers.Conv2D(32, (3, 3), activation='relu')(inputs_depth)
    conv_depth = layers.MaxPooling2D((2, 2))(conv_depth)
    conv_depth = layers.Conv2D(64, (3, 3), activation='relu')(conv_depth)
    conv_depth = layers.MaxPooling2D((2, 2))(conv_depth)
    conv_depth = layers.Conv2D(128, (3, 3), activation='relu')(conv_depth)
    conv_depth = layers.MaxPooling2D((2, 2))(conv_depth)
    flatten_depth = layers.Flatten()(conv_depth)

    # Concatenate image and depth features
    concatenated = layers.Concatenate()([flatten_img, flatten_depth])

    # Fully connected layers
    dense1 = layers.Dense(256, activation='relu')(concatenated)
    output = layers.Dense(num_classes)(dense1)

    model = models.Model(inputs=[inputs_img, inputs_depth], outputs=output)
    return model

In [ ]:
# Create and compile the model
model = create_model()
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
#history = model.fit([images_train, depths_train], labels_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model
#loss = model.evaluate([images_test, depths_test], labels_test)
#print("Test Loss:", loss)

# +
# Train the model
history = model.fit([images_train, depths_train], labels_train, batch_size=batch_size, epochs=epochs, 
                    validation_split=0.2, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
                               tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: print('\nEpoch {}: Validation MAE: {:.4f}'.format(epoch+1, logs['val_loss'])))])

# Evaluate the model
loss = model.evaluate([images_test, depths_test], labels_test)
print("Test Loss:", loss)

# Predict labels for test set
predictions = model.predict([images_test, depths_test])

# Calculate accuracy
accuracy = np.mean(np.abs(predictions - labels_test))
print("Test Accuracy (MAE):", accuracy)

# +
#print(labels_test)

# +
#print(predictions)
# -

import matplotlib.pyplot as plt
# Plot the distribution of MAE
plt.figure(figsize=(8, 6))
plt.hist(np.abs(predictions - labels_test), bins=20, edgecolor='k')
plt.title('Distribution of Absolute Error')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# +
import matplotlib.pyplot as plt

# Plot predictions vs labels_test
plt.figure(figsize=(8, 6))
plt.scatter(labels_test, predictions, color='blue')
plt.plot([min(labels_test), max(labels_test)], [min(labels_test), max(labels_test)], color='red', linestyle='--')
plt.title('Predictions vs Actual Labels')
plt.xlabel('Actual Labels')
plt.ylabel('Predictions')
plt.grid(True)
plt.show()

# -

Epoch 1/20
20/20 [==============================] - ETA: 0s - loss: 249639.4219
Epoch 1: Validation MAE: 94.0254
20/20 [==============================] - 11s 501ms/step - loss: 249639.4219 - val_loss: 94.0254
Epoch 2/20
20/20 [==============================] - ETA: 0s - loss: 84.9296
Epoch 2: Validation MAE: 53.5644
20/20 [==============================] - 11s 532ms/step - loss: 84.9296 - val_loss: 53.5644
Epoch 3/20
 9/20 [============>.................] - ETA: 6s - loss: 53.6204